current best : XG boost model , with 20 estimators , score = 73297.75 , rank = 860

Improving the model :
 - understand the data
 - design new features
 - try different preprocessing
 - try different models
 - combine multiple models

#Initialization


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
import seaborn as sb
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
!pip install -U xgboost

In [ ]:
!pip install catboost

  Using cached catboost-1.2.7-cp310-cp310-manylinux2014_x86_64.whl.metadata (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


# selecting the data file

merging 2 files to create more training data

In [ ]:
# Read the first CSV file
df1 = pd.read_csv('carpricedata.csv')  # Replace with your first CSV file path

# Read the second CSV file
df2 = pd.read_csv('train.csv')  # Replace with your second CSV file path

# Concatenate the two DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True)

selecting data file


In [ ]:
#file_path = 'test.csv' # to test
file_path = 'train.csv' # to train
data = pd.read_csv(file_path)

# train on max data
#data =merged_df

#cleaning pipeline


In [ ]:

# 1) Brand .........................................................................................................................................................................
brand_dict = {
    "Acura": 1, "Alfa": 2, "Aston": 3, "Audi": 4, "Bentley": 5, "BMW": 6,
    "Bugatti": 7, "Buick": 8, "Cadillac": 9, "Chevrolet": 10, "Chrysler": 11,
    "Dodge": 12, "Ferrari": 13, "FIAT": 14, "Ford": 15, "Genesis": 16,
    "GMC": 17, "Honda": 18, "Hummer": 19, "Hyundai": 20, "INFINITI": 21,
    "Jaguar": 22, "Jeep": 23, "Karma": 24, "Kia": 25, "Lamborghini": 26,
    "Land": 27, "Lexus": 28, "Lincoln": 29, "Lotus": 30, "Lucid": 31,
    "Maserati": 32, "Maybach": 33, "Mazda": 34, "McLaren": 35,
    "Mercedes-Benz": 36, "Mercury": 37, "MINI": 38, "Mitsubishi": 39,
    "Nissan": 40, "Plymouth": 41, "Polestar": 42, "Pontiac": 43, "Porsche": 44,
    "RAM": 45, "Rivian": 46, "Rolls-Royce": 47, "Saab": 48, "Saturn": 49,
    "Scion": 50, "smart": 51, "Subaru": 52, "Suzuki": 53, "Tesla": 54,
    "Toyota": 55, "Volkswagen": 56, "Volvo": 57
}
def map_brand_to_number(brand):
    return brand_dict.get(brand, "Unknown Brand")

# Apply the function to the 'brand' column and create a new column 'brand_number'
data['brand_num'] = data['brand'].apply(map_brand_to_number)

# 2 ) model year ....................................................................................................................................................................

# Function to assign brackets based on the year
def assign_bracket(year):
    if year < 1974:
        return "Out of range"
    return (year - 1974) // 5 + 1

# Apply the function to the 'year' column and create a new column 'year_bracket'
data['model_yr'] = data['model_year'].apply(assign_bracket)

# 3 ) milage .....................................................................................................................................................................

# Function to assign brackets based on mileage
def assign_mileage_bracket(mileage):
    if mileage < 100:
        return "Out of range"
    return (mileage - 100) // 100 + 1

# Apply the function to the 'mileage' column and create a new column 'mileage_bracket'
data['milage_num'] = data['milage'].apply(assign_mileage_bracket)

# 4) fuel ............................................................................................................................................................................


# Create a dictionary to map fuel types to labels
fuel_dict = {
    "Diesel": 1,
    "E85 Flex Fuel": 2,
    "Gasoline": 3,
    "Hybrid": 4,
    "not supported": 5,
    "Plug-In Hybrid": 6,

}

# Function to map fuel types to their corresponding labels
def map_fuel_to_label(fuel):
    return fuel_dict.get(fuel,0)

# Apply the function to the 'fuel_type' column and create a new column 'fuel_label'
data['fuel_num'] = data['fuel_type'].apply(map_fuel_to_label)

# 5) accident ......................................................................................................................................................................

def assign_accident_value(accident):
    return 1 if accident == "At least 1 accident or damage reported" else 0

# Apply the function to the 'accident' column and create a new column 'accident_flag'
data['accident_num'] = data['accident'].apply(assign_accident_value)

# 6) clean title .............................................................................................................................................................

def assign_clean_title_value(clean_title):
    if isinstance(clean_title, str):  # Check if the value is a string
        return 1 if clean_title.strip().lower() == "yes" else 0
    return 0  # If it's not a string (e.g., NaN or a number), assign 0
# Apply the function to the 'clean_title' column and create a new column 'clean_title_flag'
data['clean_title_num'] = data['clean_title'].apply(assign_clean_title_value)

data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,brand_num,model_yr,milage_num,fuel_num,accident_num,clean_title_num
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,38,7,2130,3,0,1
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,29,6,1432,3,1,1
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,10,6,1367,2,0,1
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,16,9,195,3,0,1
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,36,10,73,3,0,1


#training the model


In [ ]:
data.dropna()
y= data.price

# Create X (After completing the exercise, you can return to modify this line!)
features = ['brand_num', 'fuel_num', 'milage_num', 'model_yr', 'accident_num', 'clean_title_num']

# Select columns corresponding to features, and preview the data
X = data[features]
X.head()
#print(X.isnull().sum())

# Option 1: Drop rows with NaN values
#X = X.dropna()

# Option 2: Fill NaN values with the mean
#X = X.fillna(X.mean())

# Option 3: Use an imputer
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

#y = y[X.index]  # Align y with X

# Check again for NaN values
#print(X.isnull().sum())

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)


In [ ]:
# stack experiment , ensembling

# 1 stacking 2 same models (xgb)
'''xgb1 = XGBRegressor(n_estimators=18)
xgb1.fit(train_X, train_y)
xgb1_predictions = xgb1.predict(val_X)
xgb1_mae = mean_absolute_error(xgb1_predictions, val_y)
print('xgb1 mae : ',xgb1_mae)
xgb2 = XGBRegressor(n_estimators=18)
xgb2.fit(train_X, train_y)
xgb2_predictions = xgb2.predict(val_X)
xgb2_mae = mean_absolute_error(xgb2_predictions, val_y)
print('xgb2 mae : ',xgb2_mae)

xgbs2 = StackingRegressor(estimators=[('xgb1', xgb1), ('xgb2', xgb2)],final_estimator=LinearRegression())
xgbs2.fit(train_X, train_y)
xgbs2_predictions = xgbs2.predict(val_X)
xgbs2_mae = mean_absolute_error(xgbs2_predictions, val_y)
print('xgbs2 mae : ',xgbs2_mae)'''

# 2 stacking a logistic regression model and xgb
'''xgb1 = XGBRegressor(n_estimators=18)
xgb1.fit(train_X, train_y)
xgb1_predictions = xgb1.predict(val_X)
xgb1_mae = mean_absolute_error(xgb1_predictions, val_y)
print('xgb1 mae : ',xgb1_mae)
lr = RandomForestRegressor()
lr.fit(train_X, train_y)
lr_predictions = lr.predict(val_X)
lr_mae = mean_absolute_error(lr_predictions, val_y)
print('xgb1 mae : ',lr_mae)'''




TypeError: missing a required argument: 'y_pred'

In [ ]:
# random forest
rf= RandomForestRegressor(random_state=1)
rf.fit(train_X, train_y)
rf_predictions = rf.predict(val_X)
rf_mae = mean_absolute_error(rf_predictions, val_y)
print(rf_mae)

22760.991340066194


In [ ]:
# XG Boost
xgb = XGBRegressor(n_estimators=20)
xgb.fit(train_X, train_y)
xgb_predictions = xgb.predict(val_X)
xgb_mae = mean_absolute_error(xgb_predictions, val_y)
print(xgb_mae)

20241.9888782897


In [ ]:
# cat boost
cat = CatBoostRegressor(loss_function='RMSE')
cat.fit(train_X, train_y) #, verbose=100
pred=cat.predict(val_X)
cat_mae=mean_absolute_error(pred,val_y)
print(cat_mae)

Learning rate set to 0.089526
0:	learn: 80998.9082757	total: 23ms	remaining: 22.9s
100:	learn: 76087.9161596	total: 1.8s	remaining: 16s
200:	learn: 75355.0313775	total: 3.53s	remaining: 14s
300:	learn: 74667.2747507	total: 5.26s	remaining: 12.2s
400:	learn: 74095.1193906	total: 8.22s	remaining: 12.3s
500:	learn: 73570.5168409	total: 10.9s	remaining: 10.9s
600:	learn: 73198.7367545	total: 12.6s	remaining: 8.36s
700:	learn: 72866.1222334	total: 14.3s	remaining: 6.09s
800:	learn: 72481.0478934	total: 16s	remaining: 3.98s
900:	learn: 72166.8344236	total: 17.7s	remaining: 1.95s
999:	learn: 71773.4422533	total: 19.4s	remaining: 0us
20241.221347826267


In [ ]:
xgbs2 = StackingRegressor(estimators=[('xgb1', xgb) , ('xgb2', cat)],final_estimator=LinearRegression())
xgbs2.fit(train_X, train_y)
xgbs2_predictions = xgbs2.predict(val_X)
xgbs2_mae = mean_absolute_error(xgbs2_predictions, val_y)
print('xgbs2 mae : ',xgbs2_mae)

Streaming output truncated to the last 5000 lines.
5:	learn: 79042.7274931	total: 86ms	remaining: 14.2s
6:	learn: 78795.2483736	total: 100ms	remaining: 14.2s
7:	learn: 78595.4351203	total: 114ms	remaining: 14.1s
8:	learn: 78395.2495139	total: 128ms	remaining: 14.1s
9:	learn: 78223.1825094	total: 142ms	remaining: 14.1s
10:	learn: 78085.7996642	total: 156ms	remaining: 14s
11:	learn: 77955.6185830	total: 170ms	remaining: 14s
12:	learn: 77865.1029574	total: 189ms	remaining: 14.3s
13:	learn: 77753.7179041	total: 205ms	remaining: 14.5s
14:	learn: 77679.7343243	total: 219ms	remaining: 14.4s
15:	learn: 77598.8355116	total: 234ms	remaining: 14.4s
16:	learn: 77524.6430761	total: 248ms	remaining: 14.3s
17:	learn: 77468.8070272	total: 262ms	remaining: 14.3s
18:	learn: 77419.0997170	total: 276ms	remaining: 14.2s
19:	learn: 77381.3870822	total: 290ms	remaining: 14.2s
20:	learn: 77334.7555187	total: 304ms	remaining: 14.2s
21:	learn: 77297.4967007	total: 319ms	remaining: 14.2s
22:	learn: 77263.1543767

Results for stacking experiments

1) stacking 2 same yet individua xgb : mae(indi models)=20239 mae(stack)=20273 {performed worse}

2) stacking a linear regressor and xgb : xgbmae=20239 lrmae= , finalmae= {took too long to fit data , skipped}

3) stacking random forest and xgb : xgbmae=20239 rfmae=22808 , finalmae=20257 {worse than xgb alone}

#predicting the values


In [ ]:
dd = data[features]
prediction_data = data[['id']].copy()  # Copy 'id' column

# Make predictions for the original dataset
predictions = xgbs2.predict(dd)

# Assign predictions to the new DataFrame
prediction_data['price'] = predictions
prediction_data['price'] = prediction_data['price'].round(3)


In [ ]:
prediction_data.head()
prediction_data.shape
#Submission must have 125690 rows

(125690, 2)

#saving the output file

In [ ]:
prediction_data.to_csv('answer_stackcatxgb.csv',index=False)

#fun

In [ ]:
# visualising my current rank (826/1179)
r=830
t=1179
for i in range(0,(r//100)-1):
  print("o")
print("-")
for i in range(0,(t-r)//100):
  print('x')


o
o
o
o
o
o
o
-
x
x
x
